# Подготовка датафрейма с разметкой изображений на основании xml. файла из программы CVAT

In [ ]:
import os
import glob
import imageio
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
img_path = '/content/images'
annotations = '/content/annotation'

In [ ]:
# Загружаем изображения как массивы NumPy и добавляем их в список images
images = []
for file in glob.glob(os.path.join(img_path, '*.jpg')):
    images.append(imageio.imread(file))

<ipython-input-12-2ce9de1eb72a>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file))


In [ ]:
# Сколько у нас изображений
print('We have {} images'.format(len(images)))


We have 210 images


In [ ]:
# Функция для конвертации XML в DataFrame
def xml_to_csv(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    width = int(root.find('imagesize').find('ncols').text)
    height = int(root.find('imagesize').find('nrows').text)

    data = []
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        xmin = ymin = xmax = ymax = None

        if len(obj.findall('polygon')) > 0:
            polygon = obj.find('polygon')
            pts = polygon.findall('pt')
            x_coords = [float(pt.find('x').text) for pt in pts]
            y_coords = [float(pt.find('y').text) for pt in pts]
            xmin = min(x_coords)
            xmax = max(x_coords)
            ymin = min(y_coords)
            ymax = max(y_coords)

        data.append([filename, width, height, obj_name, xmin, ymin, xmax, ymax])

    column_names = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    df = pd.DataFrame(data, columns=column_names)
    return df

In [ ]:
# Создаем пустой DataFrame для сохранения данных
labels_df = pd.DataFrame()

In [ ]:
# Пройдем по всем XML файлам в папке с аннотациями и конвертируем их в CSV
for file in os.listdir(annotations):
    if file.endswith('.xml'):
        df = xml_to_csv(os.path.join(annotations, file))
        labels_df = pd.concat([labels_df, df], ignore_index=True)


print('Successfully converted xml to csv.')
print(labels_df)

Successfully converted xml to csv.
        filename  width  height  class    xmin    ymin    xmax    ymax
0    2 (112).jpg    854     480  break  505.52  177.12  537.08  233.02
1    2 (112).jpg    854     480  crack  126.72   39.05  548.75  456.60
2    2 (112).jpg    854     480  joint  507.45    1.20  722.75  479.80
3    2 (112).jpg    854     480  crack  632.60    2.80  725.70  352.80
4     3 (86).jpg    854     480  crack  267.44   34.29  588.16  284.40
..           ...    ...     ...    ...     ...     ...     ...     ...
919  3 (127).jpg    854     480  break  332.40  406.60  412.85  423.80
920  2 (114).jpg    854     480  break  507.34  174.97  538.88  224.42
921  2 (114).jpg    854     480  crack  126.72   37.20  555.60  456.60
922  2 (114).jpg    854     480  joint  513.20    1.90  728.10  480.00
923  2 (114).jpg    854     480  crack  661.88    2.67  726.30  296.70

[924 rows x 8 columns]


In [ ]:
# Загрузите DataFrame с аннотациями
labels_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,2 (112).jpg,854,480,break,505.52,177.12,537.08,233.02
1,2 (112).jpg,854,480,crack,126.72,39.05,548.75,456.60
2,2 (112).jpg,854,480,joint,507.45,1.20,722.75,479.80
3,2 (112).jpg,854,480,crack,632.60,2.80,725.70,352.80
4,3 (86).jpg,854,480,crack,267.44,34.29,588.16,284.40
...,...,...,...,...,...,...,...,...
919,3 (127).jpg,854,480,break,332.40,406.60,412.85,423.80
920,2 (114).jpg,854,480,break,507.34,174.97,538.88,224.42
921,2 (114).jpg,854,480,crack,126.72,37.20,555.60,456.60
922,2 (114).jpg,854,480,joint,513.20,1.90,728.10,480.00


In [ ]:
# Укажите исходные и новые размеры изображений
original_width, original_height = 854, 480
new_width, new_height = 600, 337

In [ ]:
def resize_image_and_update_coords(image_path, boxes):
    with Image.open(image_path) as img:
        img_resized = img.resize((new_width, new_height), Image.ANTIALIAS)

        # Сохраните новое изображение в нужную директорию (например, "images_resized")
        new_image_path = os.path.join("images_resized", os.path.basename(image_path))
        os.makedirs("images_resized", exist_ok=True)
        img_resized.save(new_image_path)

        # Обновите координаты bounding boxes
        scale_x = new_width / original_width
        scale_y = new_height / original_height
        boxes = boxes.copy()
        boxes['xmin'] = boxes['xmin'] * scale_x
        boxes['ymin'] = boxes['ymin'] * scale_y
        boxes['xmax'] = boxes['xmax'] * scale_x
        boxes['ymax'] = boxes['ymax'] * scale_y

        # Обновите размеры изображения в DataFrame
        boxes['width'] = new_width
        boxes['height'] = new_height
        boxes['filename'] = os.path.basename(new_image_path)

        return boxes

In [ ]:
# Обрабатываем каждое изображение и обновляем координаты
updated_frames = []
for image_file in tqdm(labels_df['filename'].unique()):
    image_path = os.path.join("images", image_file)  # Укажите путь к директории с изображениями

    # Получаем все записи для данного изображения
    image_boxes = labels_df[labels_df['filename'] == image_file]

    # Обновляем координаты bounding boxes
    updated_boxes = resize_image_and_update_coords(image_path, image_boxes)

    updated_frames.append(updated_boxes)

# Объединяем все обновленные DataFrame
updated_labels_df = pd.concat(updated_frames)

  0%|          | 0/210 [00:00<?, ?it/s]<ipython-input-54-4ea363e3c06b>:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((new_width, new_height), Image.ANTIALIAS)
100%|██████████| 210/210 [00:03<00:00, 55.38it/s]


In [ ]:
updated_labels_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,2 (112).jpg,600,337,break,355.166276,124.353000,377.339578,163.599458
1,2 (112).jpg,600,337,crack,89.030445,27.416354,385.538642,320.571250
2,2 (112).jpg,600,337,joint,356.522248,0.842500,507.786885,336.859583
3,2 (112).jpg,600,337,crack,444.449649,1.965833,509.859485,247.695000
462,2 (112).jpg,600,337,break,355.166276,124.353000,377.339578,163.599458
...,...,...,...,...,...,...,...,...
461,2 (114).jpg,600,337,crack,465.021077,1.874562,510.281030,208.308125
920,2 (114).jpg,600,337,break,356.444965,122.843521,378.604215,157.561542
921,2 (114).jpg,600,337,crack,89.030445,26.117500,390.351288,320.571250
922,2 (114).jpg,600,337,joint,360.562061,1.333958,511.545667,337.000000


In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
updated_labels_df['class'] = labelencoder.fit_transform(updated_labels_df['class'])


In [ ]:
updated_labels_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,2 (112).jpg,600,337,0,355.166276,124.353000,377.339578,163.599458
1,2 (112).jpg,600,337,1,89.030445,27.416354,385.538642,320.571250
2,2 (112).jpg,600,337,2,356.522248,0.842500,507.786885,336.859583
3,2 (112).jpg,600,337,1,444.449649,1.965833,509.859485,247.695000
462,2 (112).jpg,600,337,0,355.166276,124.353000,377.339578,163.599458
...,...,...,...,...,...,...,...,...
461,2 (114).jpg,600,337,1,465.021077,1.874562,510.281030,208.308125
920,2 (114).jpg,600,337,0,356.444965,122.843521,378.604215,157.561542
921,2 (114).jpg,600,337,1,89.030445,26.117500,390.351288,320.571250
922,2 (114).jpg,600,337,2,360.562061,1.333958,511.545667,337.000000


In [ ]:
# Сохраняем обновленный DataFrame в новый CSV-файл
updated_labels_df.to_csv("new_annotations.csv", index=False)

print("Процесс завершен. Изображения и аннотации обновлены и сохранены.")

Процесс завершен. Изображения и аннотации обновлены и сохранены.
